# *Python from Scratch LiveLessons*
## Lesson 7: Pie Factory&#8212;Object-Oriented Python
<img src="misc/pie_cover_variation_.png" width="60%"  />

API is short for Application Programming Intersface. This is how we make it so one program talks to another. We will conduct the following steps:

1. Read the API documentation for our *Factory Conveyor Belt* we will use (already exists).
1. Create callbacks.
1. Create a Mock Browser (simulation) to handle running our Factory. 
1. Run our Mock Browser.

In Lesson, 8 we will finalize our whole program by hooking up our code to the the actual web browser. By using this Mock Server we will write ???, and by breaking this up into smaller parts we simplify the approach.  

In [1]:
import jupy

In [2]:
import complete_pie

### Read the API documentation for our Factory Conveyor Belt we will use (already exists).

In [3]:
help(complete_pie.FactoryConveyorBelt)

Help on class FactoryConveyorBelt in module complete_pie:

class FactoryConveyorBelt(builtins.object)
 |  Handles holding state for pies and adding callbacks to handle operations
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add_callback(self, method, func)
 |      add a callback function for given 'method'
 |  
 |  add_pie(self, pie)
 |      add a single pie to the FactoryConveyorBelt
 |  
 |  add_pie_order(self, pie, qty)
 |      add a bunch of pies to FactoryConveyorBelt
 |  
 |  fill_pantry(self, pies, times=5)
 |      Given a list of 'pies', duplicate items times 'times', adds to self.inventory
 |  
 |  get_totals(self)
 |      shows totals a s pretty ascii-format table
 |  
 |  key_item_for_inventory(self)
 |      give current inventory as `dict`
 |  
 |  run_belt(self)
 |      run the FactoryConveyorBelt in production
 |  
 |  run_belt_test(self)
 |      simulate running the FactoryConveyorB

Below is our design of basic flow:

<img src="misc/workflow_2.png" width="60%"  />


The callbacks are functions that carry out the task at hand for each operation. The FactoryConveyorBelt holds the data. So we create an instance of FactoryConveryorBelt first:



In [4]:
from complete_pie import FactoryConveyorBelt, ApplePie
belt = FactoryConveyorBelt()

Next we create a Pie and add it to the belt:

In [5]:
pie = ApplePie(name="Example Apple Pie", recipe_path="misc/ApplePie.txt")
pie.process_recipe()
belt.fill_pantry(pie, times=10)
print(belt.get_totals())

DEBUG: restock


+------------------------------+------------------------------+------------------------------+
|            solid             |          large item          |            liquid            |
+------------------------------+------------------------------+------------------------------+
|     22 1/2 cup of flour      | 30  item of Granny Smith ... | 10  tsp of apple pie spic... |
|    5  cup of brown sugar     |                              |    2 1/2 cup of ice water    |
|       10  tsp of salt        |                              |                              |
| 5  cup of granulated suga... |                              |                              |
| 7 1/2 cup of solid shorte... |                              |                              |
+------------------------------+------------------------------+------------------------------+


In [6]:
belt.add_pie_order(pie, 3)

In [7]:
print(belt.get_totals())

+------------------------------+------------------------------+------------------------------+
|            solid             |          large item          |            liquid            |
+------------------------------+------------------------------+------------------------------+
|     20 1/4 cup of flour      | 27  item of Granny Smith ... | 9  tsp of apple pie spice... |
|   4 1/2 cup of brown sugar   |                              |    2 1/4 cup of ice water    |
|        9  tsp of salt        |                              |                              |
| 4 1/2 cup of granulated s... |                              |                              |
| 6 3/4 cup of solid shorte... |                              |                              |
+------------------------------+------------------------------+------------------------------+


The belt knows about certain methods:

In [8]:
belt.known_callback_methods

('bake', 'reload', 'oven', 'restock')

We can add a simple callback `echo()` to all of theses. Please note that all callbacks take the `callback_app` and `message` as arguments:

In [9]:
def echo_callback(callback_app, message):
    callback_app.logger.info("echo {}".format(message))

for method in belt.known_callback_methods:
    belt.add_callback(method, echo_callback)

Already created for you is the Mock Browser:

In [10]:
# %load mock_browser.py

from datetime import datetime
import random
from pie_logger import get_logger
log = get_logger()


#  socket.emit('callback', {action: 'oven', unique_pie_id: pie.unique_pie_id, heat_time: game.time.now})
#  socket.emit('callback', {action: 'bake', baketype: 'apple'});
#  socket.emit('callback', {action: 'bake', baketype: 'cherry'});
#  socket.emit('callback', {action: 'bake', baketype: 'raseberry'});
#  socket.emit('callback', {action: 'restock'});


class MockApp:
    def __init__(self, belt):
        self.logger = log
        self.belt = belt


def get_random_pie():
    return random.choice(["apple", "cherry", "raseberry"])


def simulate(belt, count, delay=5):

    callback_app = MockApp(belt)

    for callback in belt.callbacks.get("restock", []):
        callback(callback_app, {})

    for n in range(count):
        log.debug("testing bake callback")
        for callback in belt.callbacks.get('bake', []):
            bake_out = callback(callback_app, dict(baketype=get_random_pie()))
            if bake_out:
                for callback in belt.callbacks.get('oven', []):
                    callback(callback_app, dict(
                        unique_pie_id=bake_out.get('unique_pie_id'),
                        heat_time=4))
    log.debug("\n"+belt.get_totals())


In [11]:
simulate(belt, 5)

INFO: echo {}
DEBUG: testing bake callback
INFO: echo {'baketype': 'raseberry'}
DEBUG: testing bake callback
INFO: echo {'baketype': 'apple'}
DEBUG: testing bake callback
INFO: echo {'baketype': 'apple'}
DEBUG: testing bake callback
INFO: echo {'baketype': 'raseberry'}
DEBUG: testing bake callback
INFO: echo {'baketype': 'apple'}
DEBUG: 
+------------------------------+------------------------------+------------------------------+
|            solid             |          large item          |            liquid            |
+------------------------------+------------------------------+------------------------------+
|     20 1/4 cup of flour      | 27  item of Granny Smith ... | 9  tsp of apple pie spice... |
|   4 1/2 cup of brown sugar   |                              |    2 1/4 cup of ice water    |
|        9  tsp of salt        |                              |                              |
| 4 1/2 cup of granulated s... |                              |                           

Now we add the callbacks `bake_callback()` and `oven_callback()`:

In [12]:
from complete_pie import RaseberryPie, CherryPie

names = ["Bob", "Sue", "Pap", "Karen", "Brian", "Greg"]

def bake_callback(callback_app, message):
    callback_app.logger.info("bake callback")
    baketype = message['baketype']
    if baketype == 'apple':
        pie_type = ApplePie
    elif baketype == 'cherry':
        pie_type = CherryPie
    elif baketype == 'raseberry':
        pie_type = RaseberryPie
    else:
        raise Exception("unknown bake type {}".format(message['baketype']))
    pie = pie_type(name="{}'s {} Pie".format(random.choice(names), baketype.title()),
                   recipe_path="misc/ApplePie.txt")
    pie.process_recipe()
    try:
        callback_app.belt.add_pie(pie)
    except Exception as e:
        return dict(error=str(e))

    totals = callback_app.belt.get_totals()
    return dict(image_key=pie.image_key,
                totals=totals,
                name=pie.name,
                unique_pie_id=pie.unique_pie_id)

belt.add_callback("bake", bake_callback)

def oven_callback(callback_app, message):
    callback_app.logger.info("message {}".format(message))
    pie = callback_app.belt.pies[message['unique_pie_id']]
    total_time = message['heat_time'] + callback_app.belt.oven_heat_time
    totals = callback_app.belt.get_totals()
    return dict(image_key=pie.image_key,
                totals=totals,
                oven_msg="Oven Heating",
                name=pie.name,
                bake_time=pie.bake_time,
                total_time=total_time,
                unique_pie_id=pie.unique_pie_id)

belt.add_callback("oven", oven_callback)

In [13]:
simulate(belt, 5)

INFO: echo {}
DEBUG: testing bake callback
INFO: echo {'baketype': 'cherry'}
INFO: bake callback
INFO: echo {'heat_time': 4, 'unique_pie_id': 'db84e8c0-9e5a-4810-9382-9da7c0b1a86e'}
INFO: message {'heat_time': 4, 'unique_pie_id': 'db84e8c0-9e5a-4810-9382-9da7c0b1a86e'}
DEBUG: testing bake callback
INFO: echo {'baketype': 'cherry'}
INFO: bake callback
INFO: echo {'heat_time': 4, 'unique_pie_id': 'b1c49330-5427-444c-b7b1-e801b6bf33b6'}
INFO: message {'heat_time': 4, 'unique_pie_id': 'b1c49330-5427-444c-b7b1-e801b6bf33b6'}
DEBUG: testing bake callback
INFO: echo {'baketype': 'raseberry'}
INFO: bake callback
INFO: echo {'heat_time': 4, 'unique_pie_id': '89dd8b77-493b-4e18-9f30-ded81f5b046a'}
INFO: message {'heat_time': 4, 'unique_pie_id': '89dd8b77-493b-4e18-9f30-ded81f5b046a'}
DEBUG: testing bake callback
INFO: echo {'baketype': 'cherry'}
INFO: bake callback
INFO: echo {'heat_time': 4, 'unique_pie_id': '7bfde371-e240-4cbf-9d08-e887cca2f143'}
INFO: message {'heat_time': 4, 'unique_pie_id':